In [10]:
from pprint import pprint
import mlflow
from mlflow.models import infer_signature
from mlflow.entities import ViewType
from mlflow.tracking import MlflowClient
import boto3

In [ ]:
S3_BUCKET_NAME=

In [ ]:
s3 = boto3.client('s3')
vectorizer_path = '/tmp/tfidf_vectorizer.pkl'
s3.download_file(S3_BUCKET_NAME, VECTORIZER_KEY, vectorizer_path)


In [2]:
tracking_uri="http://127.0.0.1:5001"
mlflow.set_tracking_uri(tracking_uri)
client = MlflowClient(tracking_uri=tracking_uri)

In [3]:
for rm in client.search_registered_models():
    pprint(dict(rm), indent=4)

In [12]:
experiment_name = "sent-analisis-xgboost-best-models"
experiment = mlflow.get_experiment_by_name(experiment_name)
experiment

<Experiment: artifact_location='s3://mflow-artifacts-381491831545/artifacts_dest/2', creation_time=1723329149934, experiment_id='2', last_update_time=1723329149934, lifecycle_stage='active', name='sent-analisis-xgboost-best-models', tags={}>

In [6]:
print(experiment.experiment_id)

2


In [7]:
model_name = "xgboost-model"
experiment_id = experiment.experiment_id
top_n = 2
best_run = client.search_runs(
            experiment_ids=experiment_id,
            run_view_type=ViewType.ACTIVE_ONLY,
            max_results=top_n,
            order_by=["metrics.test_accuracy ASC"]
        )[0]

In [16]:
best_run

<Run: data=<RunData: metrics={'logloss': 0.5409361193588802,
 'test_accuracy': 0.7512,
 'test_recall': 0.8576,
 'val_accuracy': 0.7578285714285714,
 'val_recall': 0.8559130040779338}, params={'alpha': '0.15354745251031074',
 'colsample_bytree': '0.776396161750482',
 'eta': '0.016107725717848798',
 'eval_metric': 'logloss',
 'lambda': '2.608417599772995e-08',
 'max_depth': '6',
 'objective': 'binary:logistic',
 'subsample': '0.6613083555020394'}, tags={'mlflow.log-model.history': '[{"run_id": "bc13c5dcbc6d4d9a8ef66ebfd5d8724a", '
                             '"artifact_path": "model", "utc_time_created": '
                             '"2024-08-10 00:12:33.950712", "flavors": '
                             '{"python_function": {"loader_module": '
                             '"mlflow.xgboost", "python_version": "3.10.13", '
                             '"data": "model.xgb", "env": {"conda": '
                             '"conda.yaml", "virtualenv": "python_env.yaml"}}, '
              

In [31]:
filter_string = f"name='{model_name}'"
model_versions = client.search_model_versions(filter_string, max_results=1)

In [32]:
last_version = model_versions[0]

In [34]:
last_version.version

'1'

In [8]:
#run_id = "25d4c35e52ab442da3b68050a07a8e41"
#run_id = "3b58ce4917294f689105ce482d11a25f"
#run_id = "272a4fe00a35480fa413e77aea487432"
run_id = "a7103a297c9247cd8a034e01b0a4d490"
model_info = mlflow.register_model(
            model_uri=f"runs:/{run_id}/model",
            name="xgboost-model"
        )

Registered model 'xgboost-model' already exists. Creating a new version of this model...
2024/08/11 00:35:56 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: xgboost-model, version 3
Created version '3' of model 'xgboost-model'.


<ModelVersion: aliases=[], creation_timestamp=1723336556442, current_stage='None', description='', last_updated_timestamp=1723336556442, name='xgboost-model', run_id='272a4fe00a35480fa413e77aea487432', run_link='', source='s3://mflow-artifacts-975050210764/artifacts/2/272a4fe00a35480fa413e77aea487432/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='3'>

In [9]:
for rm in client.search_registered_models():
    pprint(dict(rm), indent=4)

{   'aliases': {},
    'creation_timestamp': 1723336443325,
    'description': '',
    'last_updated_timestamp': 1723336556442,
    'latest_versions': [   <ModelVersion: aliases=[], creation_timestamp=1723336556442, current_stage='None', description='', last_updated_timestamp=1723336556442, name='xgboost-model', run_id='272a4fe00a35480fa413e77aea487432', run_link='', source='s3://mflow-artifacts-975050210764/artifacts/2/272a4fe00a35480fa413e77aea487432/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='3'>],
    'name': 'xgboost-model',
    'tags': {}}


In [15]:
# Nombre del modelo registrado
model_name = "xgboost-model"

# Obtener todas las versiones del modelo registrado
model_versions = client.search_model_versions(f"name='{model_name}'", max_results=1)

In [18]:
model_versions[0].version

'3'

In [20]:
alias="production"

In [21]:

client.set_registered_model_alias(model_name, alias, model_versions[0].version)

In [ ]:
def promote_best_model(model_name,  tracking_uri="http://127.0.0.1:5000"):
    new_stage = "Production"
    client = MlflowClient(tracking_uri=tracking_uri)
    model_versions = client.search_model_versions(f"name='{model_name}'", 
                                                  max_results=1)
    # create "champion" alias for version 1 of model "example-model"
    client.set_registered_model_alias(model_name, alias, 1)